#### CMSC 462 Assignment 5 - Hamin Han

The following command resets the environment

In [16]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Import mongo

In [17]:
from pymongo import MongoClient
import pprint
import pymongo
import pandas as pd
import datetime
from bson.code import Code

Connect to  MongoDB server

In [18]:
try:
    client = MongoClient("mongodb://localhost:27017/")
    client.drop_database("Mongo_database")
except Exception as e:
    print(e)

Create database

In [19]:
try:
    db=client.get_database("Mongo_database")
except Exception as e:
    print(e)
 

Read CSV files and insert data into MongoDB

In [20]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
tags_df = pd.read_csv('tags.csv')

Insert data into respective collections

In [21]:
db.movies.insert_many(movies_df.to_dict('records'))
db.ratings.insert_many(ratings_df.to_dict('records'))
db.tags.insert_many(tags_df.to_dict('records'))

InsertManyResult([ObjectId('656fef2ccc057cfff4a0ce66'), ObjectId('656fef2ccc057cfff4a0ce67'), ObjectId('656fef2ccc057cfff4a0ce68'), ObjectId('656fef2ccc057cfff4a0ce69'), ObjectId('656fef2ccc057cfff4a0ce6a'), ObjectId('656fef2ccc057cfff4a0ce6b'), ObjectId('656fef2ccc057cfff4a0ce6c'), ObjectId('656fef2ccc057cfff4a0ce6d'), ObjectId('656fef2ccc057cfff4a0ce6e'), ObjectId('656fef2ccc057cfff4a0ce6f'), ObjectId('656fef2ccc057cfff4a0ce70'), ObjectId('656fef2ccc057cfff4a0ce71'), ObjectId('656fef2ccc057cfff4a0ce72'), ObjectId('656fef2ccc057cfff4a0ce73'), ObjectId('656fef2ccc057cfff4a0ce74'), ObjectId('656fef2ccc057cfff4a0ce75'), ObjectId('656fef2ccc057cfff4a0ce76'), ObjectId('656fef2ccc057cfff4a0ce77'), ObjectId('656fef2ccc057cfff4a0ce78'), ObjectId('656fef2ccc057cfff4a0ce79'), ObjectId('656fef2ccc057cfff4a0ce7a'), ObjectId('656fef2ccc057cfff4a0ce7b'), ObjectId('656fef2ccc057cfff4a0ce7c'), ObjectId('656fef2ccc057cfff4a0ce7d'), ObjectId('656fef2ccc057cfff4a0ce7e'), ObjectId('656fef2ccc057cfff4a0ce

Add five movies to the "movies" collection

In [22]:
movie_list = [
    {"movieId": 200000, "title": "The Notebook (2006)", "genres": "Romance"},
    {"movieId": 2000001, "title": "Titanic (1997)", "genres": "Romance"},
    {"movieId": 2000002, "title": "10 things I Hate About You (1999)", "genres": "Romance"},
    {"movieId": 2000003, "title": "100 Days of Summer", "genres (2009)": "Romance"},
    {"movieId": 2000004, "title": "Crazy Stupid Love (2011)", "genres": "Romance"}
]

db.movies.insert_many(movie_list)

InsertManyResult([ObjectId('656fef2ecc057cfff4a0dcc9'), ObjectId('656fef2ecc057cfff4a0dcca'), ObjectId('656fef2ecc057cfff4a0dccb'), ObjectId('656fef2ecc057cfff4a0dccc'), ObjectId('656fef2ecc057cfff4a0dccd')], acknowledged=True)

Add ratings and tags for the added movies 

In [23]:
user_id = "9595"  # Personal unique user_id

for movie in movie_list:
    movie_id = movie["movieId"]
    
    # Add ratings
    db.ratings.insert_one({"userId": user_id, "movieId": movie_id, "rating": 5})
    
    # Add tags
    db.tags.insert_one({"userId": user_id, "movieId": movie_id, "tag": "romantic"})


Use Aggregation Pipeline for Analysis

In [46]:
# Multi-stage aggregation pipeline for movies
movies_pipeline = [
     # Stage 1: Group by year and count movies per year
     {
        "$group": {
            "_id": {
                "$regexFind": {
                    "input": "$title",
                    "regex": r"\((\d{4})\)",
                    "options": "i"
                }
            },
            "movies_per_year": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "year": "$_id.captures[0]",
            "movies_per_year": 1
        }
    },

    # Stage 2: Unwind genres and count movies per genre
    #{"$match": {"tag": {"$ne": None}}},
    {"$group": {"_id": "$genres", "movies_per_genre": {"$sum": 1}}}
]

# Multi-stage aggregation pipeline for ratings
ratings_pipeline = [
    # Stage 3: Group by rating and count movies per rating
    #{"$match": {"tag": {"$ne": None}}},
    {"$group": {"_id": "$rating", "movies_per_rating": {"$sum": 1}}}
]

# Multi-stage aggregation pipeline for tags
tags_pipeline = [
    # Stage 4: Exclude movies with tag set to None and count movies tagged
    #{"$match": {"tag": {"$ne": None}}},
    {"$group": {"_id": "$movieId", "movies_tagged": {"$sum": 1}}},

    # Stage 5: Exclude movies with tag set to None and find the most popular tag
    #{"$match": {"tag": {"$ne": None}}},
    {"$group": {"_id": "$tag", "tag_count": {"$sum": 1}}},
    {"$sort": {"tag_count": -1}},
    {"$limit": 1}
]

# Execute the aggregation pipelines
movies_result = list(db.movies.aggregate(movies_pipeline))
ratings_result = list(db.ratings.aggregate(ratings_pipeline))
tags_result = list(db.tags.aggregate(tags_pipeline))

# Print the results
print("Movies Result:")
print(movies_result)

print("\nRatings Result:")
print(ratings_result)

print("\nTags Result:")
print(tags_result)

Movies Result:
[{'_id': None, 'movies_per_genre': 2731}]

Ratings Result:
[{'_id': 5.0, 'movies_per_rating': 13216}, {'_id': 1.0, 'movies_per_rating': 2811}, {'_id': 2.5, 'movies_per_rating': 5550}, {'_id': 1.5, 'movies_per_rating': 1791}, {'_id': 3.0, 'movies_per_rating': 20047}, {'_id': 3.5, 'movies_per_rating': 13136}, {'_id': 4.5, 'movies_per_rating': 8551}, {'_id': 4.0, 'movies_per_rating': 26818}, {'_id': 2.0, 'movies_per_rating': 7551}, {'_id': 0.5, 'movies_per_rating': 1370}]

Tags Result:
[{'_id': None, 'tag_count': 1577}]
